# Hidden Markov Models (HMMs)

A popular choice of state space model (SSM) in time-series applications is the <b>hidden Markov model (HMM)</b>. In this notebook, we will provide an introduction on how to represent an HMM, how to perform posterior inference with this model, and how to estimate its parameters. 

Let's first import the necessary modules:

In [1]:
# Widget to manipulate plots in Jupyter notebooks
%matplotlib widget 

import matplotlib.pyplot as plt # For general plotting

import pandas as pd

# GPU suitable form of NumPy, also works on the CPU
from jax import numpy as jnp
from jax import random as jr
from jax import vmap

# JAX library for SSMs
from dynamax.hidden_markov_model import CategoricalHMM

jnp.set_printoptions(suppress=True)

# Set seed to generate reproducible "pseudo-randomness"
key = jr.PRNGKey(7)

plt.rc('font', size=22)          # controls default text sizes
plt.rc('axes', titlesize=18)     # fontsize of the axes title
plt.rc('axes', labelsize=18)     # fontsize of the x and y labels
plt.rc('xtick', labelsize=14)    # fontsize of the tick labels
plt.rc('ytick', labelsize=14)    # fontsize of the tick labels
plt.rc('legend', fontsize=16)    # legend fontsize
plt.rc('figure', titlesize=22)   # fontsize of the figure title

## Markov Chains

Markov chains are sequential models that assume an observation $\mathbf{x}_t$ at the current timestep $t$ is a <b>sufficient statistic</b> to predict the future:

\begin{equation*}
p(\mathbf{x}_{t+\tau} \, | \, \mathbf{x}_t, \mathbf{x}_{1:t-1}) = p(\mathbf{x}_{t+\tau} \, | \, \mathbf{x}_t),
\end{equation*}

for $\tau \geq 0$. In other words, given the present $\mathbf{x}_t$, the future is independent of the past. This is known as the <b>Markov assumption</b>. As a result, the joint distribution of a finite length sequence, with duration $T$, can be expressed as:

\begin{equation*}
p(\mathbf{x}_{1:T}) = p(\mathbf{x}_1) p(\mathbf{x}_2 \, | \, \mathbf{x}_1) p(\mathbf{x}_3 \, | \, \mathbf{x}_2) \ldots = p(\mathbf{x}_1) \prod_{t=2}^T p(\mathbf{x}_t \, | \, \mathbf{x}_{t-1}).
\end{equation*}


## State-Space Models (SSMs)

SSMs are <b>partially observed Markov chains</b>, which tells us two things about the dynamical system we are modeling:
1. The state of this dynamical system is only "partially observable", meaning there is a hidden variable, $\mathbf{z}_t \in \mathbb{R}^{N_z}$, that is assumed to be responsible for generating any observations, $\mathbf{x}_t \in \mathbb{R}^{N_x}$, about the system's behavior.
2. The latent state's dynamics, $\mathbf{z}_t \in \mathbb{R}^{N_z}$, adhere to the Markov assumption.

Please refer to the [Kalman filter (KF) notebook](kalman_filter.ipynb#Posterior-Inference-in-SSMs) for a brief on the general problem of posterior inference in SSMs. As described in the KF notebook, we often assume that the observations are conditionally independent of one another, given $\mathbf{z}_t$, as opposed to being Markovian in their dependencies. An SSM can thus be described by the following joint distribution:

\begin{equation*}
p(\mathbf{x}_{1:T}, \mathbf{z}_{1:T}) = \left[ p(\mathbf{z}_1) \prod_{t=2}^T p(\mathbf{z}_t \, | \, \mathbf{z}_{t-1}) \right] \left[ \prod_{t=1}^T p(\mathbf{x}_t \, | \, \mathbf{z}_{t}) \right],
\end{equation*}

where $p(\mathbf{z}_1)$ is the prior distribution, $p(\mathbf{z}_t \, | \, \mathbf{z}_{t-1})$ is the transition model, and $p(\mathbf{x}_t \, | \, \mathbf{z}_{t})$ is the observation model.

## HMM Representation

HMMs are SSMs with <b>discrete</b> hidden variables, $z_t \in \{1, \ldots, K\}$. This is in contrast to linear Gaussian SSMs or linear dynamical systems, which have continuous latent variables (as described in the [KF example](kalman_filter.ipynb#Posterior-Inference-in-SSMs)). Note that the observations can be discrete, $x_t \in \{1, \ldots, N_x\}$, continuous, $\mathbf{x}_t \in \mathbb{R}^{N_x}$, or a mix of both.

Let us now walk through specifying each of the distributions for this generative model.

### State Transition Model

First, the <b>initial state distribution</b> can be written as:

\begin{equation*}
p(z_1 = j) = \pi_j,
\end{equation*}

for an arbitrary state $j$. We denote $\boldsymbol{\pi}$ as a categorical distribution over the $K$ states of the HMM.

The <b>transition model</b> can then be denoted as:

\begin{equation*}
p(z_t = j | z_{t-1} = i) = A_{ij},
\end{equation*}

where $\mathbf{A}$ is the $K \times K$ transition matrix (a row stochastic matrix, i.e., each row sums to one).

### Observation Model

Lastly, the <b>observation likelihood</b>, $p(\mathbf{x}_t \, | \, z_{t} = j)$, is represented differently depending on the type of observation data, $\mathbf{x}_t$. For discrete observations or <i>emissions</i>, $x_t \in \{1, \ldots, N_x\}$, we could utilize:

\begin{equation*}
p(x_t = k \, | \, z_t = j) = B_{jk},
\end{equation*}

with $\mathbf{B}$ as the $K \times N_x$ observation matrix containing the emission probabilities. A similar form could be written for Bernoulli observations. For $D$ discrete observations per time step, or $D$ <i>number of emissions</i>, a factorial model could be adopted:

\begin{equation*}
p(\mathbf{x}_t \, | \, z_t = j) = \prod_{d=1}^D \text{Cat}(x_{td} \, | \, \mathbf{B}_{d,j,:}),
\end{equation*}

where $x_{td}$ is the $d^{th}$ observation at time $t$, and $\mathbf{B}_{d,j,:}$ is the $j^{th}$ row of the emission probabilities matrix for observation $d$.

Likewise, if $\mathbf{x}_t \in \mathbb{R}^{N_x}$ is continuous, then a common choice of likelihood model is the multivariate Gaussian distribution:

\begin{equation*}
p(\mathbf{x}_t \, | \, z_t = j) = \mathcal{N}(\mathbf{x}_t \, | \, \boldsymbol{\mu}_j, \boldsymbol{\Sigma}_j),
\end{equation*}

where $\boldsymbol{\mu}_j \in \mathbb{R}^{N_x}$ and $\boldsymbol{\Sigma}_j \in \mathbb{R}^{N_x \times N_x}$ are the mean and covariance matrix associated with the Gaussian distribution for hidden state $j$.